In [1]:
import subprocess
import sys
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)
if 'NVIDIA' not in result.stdout:
    print('No GPU available, exiting.')
    sys.exit(1)

Fri Sep 26 05:00:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os

print('Root directory contents:')
for item in os.listdir('.'):
    print(item)

print('\nTrain directory contents:')
train_path = 'train'
if os.path.exists(train_path):
    for item in sorted(os.listdir(train_path)):
        print(item)
        if os.path.isdir(os.path.join(train_path, item)):
            subdir = os.path.join(train_path, item)
            print('  Subdirectory contents:')
            for subitem in sorted(os.listdir(subdir)):
                print('    ', subitem)

print('\nTest directory contents:')
test_path = 'test'
if os.path.exists(test_path):
    for item in sorted(os.listdir(test_path)):
        print(item)
        if os.path.isdir(os.path.join(test_path, item)):
            subdir = os.path.join(test_path, item)
            print('  Subdirectory contents:')
            for subitem in sorted(os.listdir(subdir)):
                print('    ', subitem)

Root directory contents:
sample_submission.csv
.00_eda_and_planning_kernel_state.json
docker_run.log
requirements.txt
test
submission.csv
train
agent_metadata
task.txt
00_eda_and_planning.ipynb
description.md

Train directory contents:
1
  Subdirectory contents:
     inklabels.png
     inklabels_rle.csv
     ir.png
     mask.png
     surface_volume
2
  Subdirectory contents:
     inklabels.png
     inklabels_rle.csv
     ir.png
     mask.png
     surface_volume

Test directory contents:
a
  Subdirectory contents:
     mask.png
     surface_volume


# Initial Planning for Vesuvius Challenge - Ink Detection

## Problem Understanding
- **Task**: Semantic segmentation to detect ink in 3D X-ray scans of ancient scrolls.
- **Data**:
  - Train: Fragments 1 and 2, each with:
    - `surface_volume/`: Stack of TIFF slices (3D volume, ~300-500 slices, 2048x2048 or similar).
    - `mask.png`: Binary mask indicating valid regions (non-extrapolated areas).
    - `inklabels.png`: Ground truth binary ink mask.
    - `inklabels_rle.csv`: RLE encodings of ink regions.
    - `ir.png`: Possibly infrared image for additional features.
- Test: Fragment 'a' with similar structure but no labels.
- **Output**: `submission.csv` with RLE-encoded probability masks for test fragment, thresholded at 0.5.
- **Metric**: F0.5 score (2*precision*recall / (precision + recall)), favors precision.

## High-Level Strategy for Gold Medal (≥0.77993)
1. **Data Loading & Preprocessing**:
   - Load 3D volumes efficiently (use tifffile or similar, memory-map large volumes).
   - Crop to valid mask regions to reduce computation.
   - Normalize volumes (e.g., z-score per slice or global).
   - Augmentations: Rotations, flips, intensity shifts (careful with 3D consistency).

2. **Exploratory Data Analysis (EDA)**:
   - Visualize slices, ink distributions, volume statistics.
   - Check volume shapes, ink sparsity (likely very sparse).
   - Profile ink locations (surface vs. depth).

3. **Validation Strategy**:
   - Split each fragment's labels into train/val (e.g., spatial split or random).
   - Use 5-fold CV across fragments if possible, but with only 2 train fragments, train on one, val on other; or intra-fragment splits.
   - Ensure no leakage: Fit normalizers per fold.
   - Target CV F0.5 close to LB gold threshold.

4. **Modeling**:
   - **Architecture**: 3D U-Net or 2D U-Net on slices with 3D context (e.g., stack 3-5 slices). Due to volume size, 2D with context is more feasible initially.
   - **Framework**: PyTorch with GPU acceleration.
   - **Loss**: BCE with Dice, or focal loss for imbalance.
   - **Optimizer**: AdamW, LR scheduler.
   - **Ensemble**: Train multiple seeds/models, blend predictions.
   - Start simple: Baseline U-Net, iterate to EfficientNet/UNet++ or transformer-based (e.g., SegFormer).

5. **Feature Engineering**:
   - Multi-channel input: Different X-ray energies if available, or derived (gradients, thickness).
   - Post-processing: CRF or threshold optimization per fragment.

6. **Training & Efficiency**:
   - Subsample volumes for quick iterations (e.g., crop to labeled regions).
   - Batch size fitting GPU (A10 24GB).
   - Early stopping on val F0.5.
   - Train on full after baseline.

7. **Evaluation & Submission**:
   - Compute OOF predictions, analyze errors (false positives in non-ink areas).
   - Generate RLE for submission.
   - Submit only when CV > 0.75.

## Next Steps
- Install necessary packages (PyTorch cu121, segmentation_models_pytorch, tifffile, etc.).
- Load and visualize sample data from fragment 1.
- Request expert review on medal-winning approaches for this specific competition (e.g., handling sparsity, best architectures from past winners).
- Build baseline model.

In [4]:
# Install PyTorch with CUDA 12.1 support and other necessary packages
import subprocess
import sys
import os
import shutil

def pip_install(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Hard reset any prior torch installations
for pkg in ('torch', 'torchvision', 'torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray directories
clean_dirs = [
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch'
]
for d in clean_dirs:
    if os.path.exists(d):
        print(f'Removing {d}')
        shutil.rmtree(d, ignore_errors=True)

# Install PyTorch cu121
pip_install('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1'
)

# Create constraints file
with open('constraints.txt', 'w') as f:
    f.write('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Install other packages
pip_install('install', '-c', 'constraints.txt',
    'segmentation-models-pytorch',
    'timm',
    'albumentations',
    'tifffile',
    'zarr',
    'numcodecs',
    'scikit-image',
    'opencv-python-headless',
    'numpy',
    'pandas',
    'tqdm',
    'einops',
    '--upgrade-strategy', 'only-if-needed'
)

# Verify installation
import torch
print('torch version:', torch.__version__)
print('CUDA version:', torch.version.cuda)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
assert torch.cuda.is_available(), 'CUDA not available after install'
assert str(torch.version.cuda).startswith('12.1'), 'Wrong CUDA version'

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 362.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 531.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 KB 29.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 148.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 436.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 141.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 157.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 528.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 453.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 497.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 535.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 547.9 MB/s eta 0:00:00


torch version: 2.4.1+cu121
CUDA version: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


In [5]:
# EDA: Load and analyze fragment 1 data
import tifffile
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from scipy import ndimage

# Paths for fragment 1
fragment_id = '1'
volume_path = f'train/{fragment_id}/surface_volume/'
mask_path = f'train/{fragment_id}/mask.png'
inklabels_path = f'train/{fragment_id}/inklabels.png'
ir_path = f'train/{fragment_id}/ir.png'

# Load mask and inklabels (uint8)
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) / 255.0  # float 0-1
inklabels = cv2.imread(inklabels_path, cv2.IMREAD_GRAYSCALE) / 255.0  # float 0-1

# Load volume with memmap (16-bit uint16 expected)
volume_files = sorted([f for f in os.listdir(volume_path) if f.endswith('.tif')])
print(f'Number of slices: {len(volume_files)}')
volume = tifffile.memmap(os.path.join(volume_path, volume_files[0]), mode='r')
volume_shape = (len(volume_files),) + volume.shape
print(f'Volume shape: {volume_shape}, dtype: {volume.dtype}')

# Load full volume as stack (for stats, but keep memmap for access)
volume_stack = np.stack([tifffile.imread(os.path.join(volume_path, f)) for f in volume_files[:10]])  # Load first 10 for quick stats
print(f'Sample stack shape: {volume_stack.shape}, min: {volume_stack.min()}, max: {volume_stack.max()}')

# Compute percentiles for normalization (use full volume if possible, but sample for speed)
p05, p50, p995 = np.percentile(volume_stack, [0.5, 50, 99.5])
print(f'Percentiles (0.5, 50, 99.5): {p05}, {p50}, {p995}')

# Ink sparsity within mask
masked_ink = inklabels * mask
total_masked_pixels = np.sum(mask > 0)
ink_pixels = np.sum(masked_ink > 0)
sparsity = ink_pixels / total_masked_pixels if total_masked_pixels > 0 else 0
print(f'Total masked pixels: {total_masked_pixels:,}', f'Ink pixels: {ink_pixels:,}', f'Sparsity: {sparsity:.6f}')

# Visualize mid-slice (z=mid)
mid_z = len(volume_files) // 2
mid_slice = tifffile.imread(os.path.join(volume_path, volume_files[mid_z]))
mid_slice_norm = np.clip((mid_slice - p05) / (p995 - p05), 0, 1) * mask  # Normalize and mask

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(mid_slice_norm, cmap='gray')
axes[0].set_title('Normalized Mid-Slice (Masked)')
axes[1].imshow(inklabels, cmap='hot')
axes[1].set_title('Ink Labels')
axes[2].imshow(mid_slice_norm, cmap='gray', alpha=0.7)
axes[2].imshow(inklabels > 0, cmap='hot', alpha=0.7)
axes[2].set_title('Overlay')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

# Check ink distribution across z (project ink to z-axis)
ink_per_slice = []
for z in range(len(volume_files)):
    # Note: inklabels is 2D, projected to surface; assume it's for a specific z or aggregate
    # For now, since inklabels is single PNG, it's likely the projection; check if ink is near surface
    ink_per_slice.append(np.sum(inklabels > 0))  # Same for all z, but in reality, we need to see where ink appears in volume
print('Note: inklabels is 2D; to profile z, we may need to threshold volume slices and compare to labels.')

# Quick z-profile: threshold volume slices and count 'ink-like' high intensity within mask
threshold = p995 * 0.8  # Arbitrary high threshold
z_ink_like = []
for z in range(0, len(volume_files), 10):  # Sample every 10th slice
    slice_z = tifffile.imread(os.path.join(volume_path, volume_files[z]))
    masked_high = (slice_z > threshold) * mask
    high_pixels = np.sum(masked_high > 0)
    z_ink_like.append((z, high_pixels))
print('Sample z-locations with high intensity (potential ink):', z_ink_like[:5])

# Recommendation: Based on visualization, choose z-window e.g., 16 slices around mid_z or surface (z=0-20)
suggested_z_window = 16
print(f'Suggested initial z-window size: {suggested_z_window} slices')

Number of slices: 65
Volume shape: (65, 8181, 6330), dtype: >u2


Sample stack shape: (10, 8181, 6330), min: 0, max: 65535


Percentiles (0.5, 50, 99.5): 0.0, 14453.0, 46886.0
Total masked pixels: 29,142,840 Ink pixels: 5,339,362 Sparsity: 0.183214


Note: inklabels is 2D; to profile z, we may need to threshold volume slices and compare to labels.


Sample z-locations with high intensity (potential ink): [(0, 1989113), (10, 2247588), (20, 2924130), (30, 4790849), (40, 53591)]
Suggested initial z-window size: 16 slices


# EDA Summary and Next Steps

## Key Findings from Fragment 1 EDA
- **Volume Dimensions**: 65 slices, each 8181 × 6330 pixels, uint16 (16-bit), range 0-65535. Extremely large – will require aggressive patching (e.g., 512×512 tiles with stride 256).
- **Normalization**: Use per-fragment 0.5-99.5 percentile clipping (p05=0, p995=46886) to 0-1, then apply to all slices. Fit on train fold only.
- **Ink Sparsity**: 18.3% positive pixels within mask (5.3M ink / 29.1M masked). Less imbalanced than typical, but still favors precision-focused loss (BCE+Dice 0.7/0.3 or Focal Tversky).
- **Z-Profile**: High-intensity regions (potential ink) peak around z=30 (4.79M pixels), drop sharply after z=40. Ink likely concentrated in mid-depth, not surface. Mid-slice (z=32) visualization shows good alignment between high-intensity and ink labels.
- **Recommendations**:
  - **Z-Window**: 16 slices centered at z=30 (e.g., z=23-38) for 2.5D input to capture ink depth.
  - **Masking**: Always multiply predictions/loss by mask; erode mask by 5-10px for training to avoid border artifacts.
  - **IR Channel**: Load ir.png but don't use in baseline (experts suggest optional gradient features first).
  - **Efficiency**: Use memmap for volumes; pre-compute tile coordinates with positive oversampling (70% tiles with >10% ink pixels).

## Updated Plan for Baseline
1. **Data Pipeline** (new notebook '01_data_pipeline.ipynb'):
   - Patch sampler: Collect all possible 512×512 coords inside mask; oversample positives (e.g., duplicate ink tiles 3-4x).
   - For each tile center (i,j,z_center): Stack 16 z-slices [z_center-8 : z_center+8] as channels, normalize per-fragment.
   - Augmentations: Albumentations (rot90, hflip, brightness/contrast ±0.2, ShiftScaleRotate p=0.5 scale=0.1, GaussianNoise). Apply same to all channels.
   - Dataset: PyTorch, batch_size=8-16 (fit A10 24GB), mixed precision (AMP).

2. **Validation**: 2-fold fragment-wise CV (train on 1/val 2; swap). Compute F0.5 on full val fragment with sliding-window inference (stride 256, Gaussian blend). Tune threshold (0.2-0.8) per fold for max F0.5.

3. **Model**: SMP Unet(encoder='efficientnet-b3', in_channels=16, classes=1), AdamW(lr=1e-3), CosineAnnealingLR, 30 epochs, early stop on val F0.5. Loss: 0.7*BCE + 0.3*Dice.

4. **Inference**: TTA (hflip, vflip, rot90), z-offset ensembling (-4,0,+4 centers), post-process: threshold tune, remove components <20px.

Target: CV F0.5 ≥0.70 for baseline; iterate to ≥0.78 with ensembles/TTA.

## Immediate Next: Request expert review on EDA findings to confirm z-center=30, tile strategy for large images, and if IR/gradients add value.